In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from pyvis.network import Network
from concurrent.futures import ThreadPoolExecutor, as_completed
import networkx as nx
import logging
import os
import glob
from tqdm.notebook import tqdm
import pymaid
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
catmaid_url = 'https://l1em.catmaid.virtualflybrain.org'
http_user = None
http_password = None
project_id = 1

rm = pymaid.CatmaidInstance(catmaid_url, http_user, http_password, project_id)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


# Neural structures
Load and build the structure of particular neurons

In [3]:
class simplified_structure:
    def __init__(self, neuron:pymaid.CatmaidNeuronList, silence=False):
        self.silence = silence
        if silence:
            logging.getLogger('pymaid').setLevel(logging.ERROR)
        self.neuron:pymaid.CatmaidNeuronList = neuron
        self.nodes:nx.MultiDiGraph = None
        # названия в формате id_обекта, если знак id положителен, то это нода скелета, если отрицателен то это коннектор
        self.build_structure()
        self.add_connectors_to_graph()
        self.simplify_directed_graph()

    def build_structure(self):
        nodes = self.neuron.nodes
        graph = nx.MultiDiGraph()
        for idt, parent, ntype in zip(nodes['node_id'], nodes['parent_id'], nodes['type']):
            # сома это root
            if (a:=idt >= 0):
                graph.add_node(idt, type = ntype) 
            if (b:=parent >= 0):
                graph.add_node(parent, type = ntype)
            if a and b:
                graph.add_edge(idt, parent, nodes_inside = [])
        self.nodes = graph

    def add_connectors_to_graph(self):
        connectors = pymaid.get_connectors(self.neuron)
        for idt, ctype in zip(connectors['connector_id'], connectors['type']):
            cID = -idt
            q = self.neuron.connectors[self.neuron.connectors['connector_id'] == idt]
            self.nodes.add_node(cID, type = ctype)
            for node_id in q['node_id']:
                if ctype == 'Postsynaptic':
                    self.nodes.add_edge(cID, node_id, nodes_inside = [])
                elif ctype == 'Presynaptic':
                    self.nodes.add_edge(node_id, cID, nodes_inside = [])
                else:
                    if not self.silence:
                        print(f"finded connector {idt} of type {ctype}")
                    self.nodes.add_edge(node_id, cID, nodes_inside = [])
                    self.nodes.add_edge(cID, node_id, nodes_inside = [])

    def simplify_directed_graph(self):
        def keep_nodes(graph, vid):
            return graph.nodes(True)[vid]['type'] == 'root' or vid < 0
        G = self.nodes
        original = len(self.nodes)
        while True:
            # Находим все вершины с in-degree=1 и out-degree=1
            nodes_to_remove = [
                node for node in G.nodes() 
                if G.in_degree(node) == 1 and G.out_degree(node) == 1 and not keep_nodes(G, node)
            ]
            
            if not nodes_to_remove:
                break # Если таких вершин нет, завершаем

            for node in nodes_to_remove:
                # Если узел уже был удален на предыдущей итерации этого же цикла, пропускаем
                if node not in G: 
                    continue

                # Получаем единственного предшественника и преемника
                # NetworkX гарантирует, что list(predecessors/successors) вернет один элемент,
                # если степень равна 1.
                u = list(G.predecessors(node))
                v = list(G.successors(node))

                if len(u) != 1 or len(v) != 1:
                    raise Exception('Этот эксепшен не должен никогда вызватся, но он вызвался и значит что то пошло не так')
                u = u[0]
                v = v[0]

                nodes_inside = sum((edge[-1]['nodes_inside'] for edge in G.edges(node, data = True)), [])
                    
                if u != v:
                    G.add_edge(u, v, nodes_inside = [node] + nodes_inside)

                G.remove_node(node)

        after = len(self.nodes)
        if not self.silence:
            print('removed', original - after, 'nodes.', f'Efficiency: {round(100*(1 - after/original), 1)}%')
    
    def save_as_nx_graph(self, path):
        nx.write_gml(self.nodes, path)

    def save_as_pyvis_html(self, path):
        net = Network(notebook = False, directed = True)
        net.from_nx(self.nodes)
        for node in net.nodes:
            node['label'] = node['type']
            if node['type'] == 'root':
                node['size'] = 30
            if node['id'] < 0:
                node['color'] = 'orange'
                node['size'] = 5

        net.show_buttons(filter_=['physics'])
        net.save_graph(path)

In [ ]:
neurons = pymaid.find_neurons()
all_skids = neurons.skeleton_id

for skid in tqdm(all_skids):
    try:
        neuron = pymaid.get_neuron(skid)
        structure = simplified_structure(neuron, silence=True)
        out_path = f'./neurons/{neuron.id}.gml'
        structure.save_as_nx_graph(out_path)

    except Exception as e:
        print(f"FAIL {skid}: {e}")

In [ ]:
# MULTICORE

# neurons = pymaid.find_neurons()
# all_skids = neurons.skeleton_id


# def process_neuron(skid):
#     try:
#         neuron = pymaid.get_neuron(skid)
#         structure = simplified_structure(neuron, silence=True)
#         out_path = f'./neurons/{neuron.id}.gml'
#         structure.save_as_nx_graph(out_path)
#         return f"OK {neuron.id}"
#     except Exception as e:
#         return f"FAIL {skid}: {e}"

# # Запуск в 72 потоках
# threads = 72
# with ThreadPoolExecutor(max_workers=threads) as executor:
#     futures = [executor.submit(process_neuron, skid) for skid in all_skids]

#     for future in tqdm(as_completed(futures), total=len(futures), desc="Processing neurons"):
#         print(future.result())

# Full Graph
Build the full graph out of all the neurons.

In [4]:
class composed_network:
    def __init__(self, path, skids=[]):
        """
        path: str - path to the folder with all neurons subgraphs
        skids: list(int) - skids to be used in graph construction (use all if empty [])
        """
        self.paths = [os.path.abspath(f) for f in glob.glob(os.path.join(path, "*.gml"))]

        # If skids is not empty, keep only files whose base name (without extension) matches a skid
        if skids:
            skids_str = {str(skid) for skid in skids}  # convert to string for filename matching
            self.paths = [p for p in self.paths if os.path.splitext(os.path.basename(p))[0] in skids_str]
        
        self.graphs = {}
        for path in tqdm(self.paths, desc="Loading neuron graphs"):
            self.graphs[path] = nx.read_gml(path)
            for node_id, attr_dict in self.graphs[path].nodes(True):
                filename = os.path.basename(path)
                attr_dict['owner'] = filename

        self.combined_graph = nx.compose_all(self.graphs.values())

    
    def save_as_nx_graph(self, path):
        nx.write_gml(self.combined_graph, path)

    
    def save_as_pyvis_html(self, path, colors:dict = None):
        net = Network(notebook=False, directed=True)
        net.from_nx(self.combined_graph)
        net.show_buttons(filter_=['physics'])
        for node in net.nodes:
            if node['type'] == 'root':
                node['size'] = 30
            if int(node['id']) < 0:
                node['shape'] = 'square'
                node['size'] = 5
                node['color'] = 'orange'
                if node['type'] not in ('Postsynaptic', 'Presynaptic'):
                    node['label'] = node['type']
                else:
                    node['label'] = None
            else:
                node['label'] = node['type']
                    
        if colors:
            for node in net.nodes:
                if int(node['id']) > 0:
                    if node['owner'] in colors:
                        node['color'] = colors[node['owner']]
        net.save_graph(path)

In [ ]:
network_5k = composed_network('./neurons')
network_5k.save_as_nx_graph('./complete_graph(5k).gml')

In [5]:
skids_3k = pymaid.get_skids_by_annotation('mw brain and inputs')

network_3k = composed_network('./neurons', skids=skids_3k)
network_3k.save_as_nx_graph('./complete_graph(3k).gml')

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


Loading neuron graphs:   0%|          | 0/3016 [00:00<?, ?it/s]

# Metadata Table
2 methods for table build are used:
* manual - may be faster, but not all the features are present
* automatic catmaid - slower and only in pickle, but all the data is present

In [5]:
logging.getLogger('pymaid').setLevel(logging.ERROR)
warnings.simplefilter(action="ignore", category=FutureWarning)

neurons = pymaid.find_neurons()
all_skids = neurons.skeleton_id

# Containers for metadata
neurons_metadata = []
nodes_metadata = []

for skid in tqdm(all_skids):
    try:
        neuron = pymaid.get_neuron(skid)
        
        # --- Neuron-level metadata ---
        neuron_metadata = {
            "neuron_id": neuron.id,
            "name": neuron.name,
            "type": neuron.type,
            "n_nodes": neuron.n_nodes,
            "n_connectors": neuron.n_connectors,
            "n_branches": neuron.n_branches,
            "n_leafs": neuron.n_leafs,
            "cable_length": neuron.cable_length,
            "annotation": neuron.annotations
        }
        neurons_metadata.append(neuron_metadata)

        # --- Node-level metadata ---
        nodes = neuron.nodes[["node_id", "x", "y", "z", "radius", "type"]].copy()
        nodes["neuron_id"] = neuron.id  # link to parent neuron

        # Fix radius: None if NaN or negative
        nodes["radius"] = nodes["radius"].apply(
            lambda r: None if pd.isna(r) or r <= 0 else r
        )

        # --- Connector metadata ---
        connectors = pymaid.get_connectors(neuron)[["connector_id", "x", "y", "z", "type"]].copy()
        connectors = connectors.rename(columns={"connector_id": "node_id"})
        connectors["radius"] = None  # no radius for connectors
        connectors["neuron_id"] = neuron.id  # link to parent neuron

        # Merge nodes + connectors into one table
        node_info = pd.concat([nodes, connectors], ignore_index=True)

        nodes_metadata.append(node_info)

    except Exception as e:
        print(f"FAIL {skid}: {e}")

# Convert lists to DataFrames
neurons_metadata = pd.DataFrame(neurons_metadata)
nodes_metadata = pd.concat(nodes_metadata, ignore_index=True)

# Save results to CSV
neurons_metadata.to_csv("Metadata_Neurons(manual).csv", index=False)
nodes_metadata.to_csv("Metadata_Nodes(manual).csv", index=False)

Your search parameters will retrieve ALL neurons in the dataset. Proceed? [Y/N]  Y


Make nrn:   0%|          | 0/5013 [00:00<?, ?it/s]

  0%|          | 0/5013 [00:00<?, ?it/s]

In [6]:
neurons_metadata

,neuron_id,name,type,n_nodes,n_connectors,n_branches,n_leafs,cable_length,annotation
0,7766016,H-shaped _a1l,CatmaidNeuron,5745,374,138,141,620329.875000,"[YYCWMLEH, 4_level-4_clusterID-22_signal-flow_..."
1,19431430,SOG IN left; PN into SOG very minor AL connect...,CatmaidNeuron,3029,215,129,136,334532.593750,"[RGPN_inputs_T3_sens, fragment, antennal lobe ..."
2,15564807,AN-R-Sens-B1-AVa-22,CatmaidNeuron,800,34,7,8,69523.359375,"[monosynaptic to IPC-like and Se0, Miroschniko..."
3,17383431,BAlp_ant ascending dendrite left,CatmaidNeuron,896,109,64,70,205249.703125,"[1_level-1_clusterID-1_signal-flow_-0.067, 7_l..."
4,7782409,Drunken-4 a1l,CatmaidNeuron,2749,106,39,41,265065.562500,"[CCWA09Input, A1L Interneurons, sw;idev3, sw;p..."
...,...,...,...,...,...,...,...,...,...
5008,17629170,BAlp ant s 8 Right,CatmaidNeuron,298,0,1,2,29098.191406,"[AM_published, Andrade et al. 2019, ND_all pub..."
5009,2637812,A02n_a1r Pseudolooper-4,CatmaidNeuron,1952,106,76,81,198657.343750,"[RF PMNs for Brandon, RF PMN_MN for Brandon, l..."
5010,3882998,CP contra to SEZ left; MB1: incomplete neuron ...,CatmaidNeuron,2528,58,33,34,298192.718750,"[mw dSEZ, 8_level-4_clusterID-16_signal-flow_-..."
5011,15679484,MN-L-Sens-B2-VM-08,CatmaidNeuron,851,30,5,6,70720.140625,"[Miroschnikow et al inputs and outputs, AM_pub..."


In [7]:
nodes_metadata

,node_id,x,y,z,radius,type,neuron_id
0,63947138,64266.699219,82094.101562,144000.0,NaN,end,7766016
1,63343843,39396.601562,65402.898438,61200.0,NaN,slab,7766016
2,251206,24783.599609,39288.199219,52550.0,NaN,slab,7766016
3,1763345,21842.400391,38372.398438,56200.0,NaN,slab,7766016
4,2887324,20067.800781,39246.398438,60150.0,NaN,end,7766016
...,...,...,...,...,...,...,...
11983497,21101105,81924.200000,21705.600000,46100.0,NaN,Presynaptic,16629757
11983498,21197220,65702.000000,35507.200000,39050.0,NaN,Presynaptic,16629757
11983499,21203636,82855.200000,36328.000000,41450.0,NaN,Presynaptic,16629757
11983500,21480789,79370.600000,30677.400000,41400.0,NaN,Presynaptic,16629757


In [8]:
# Build automatically using catmaid
neurons_df = pymaid.find_neurons().to_dataframe()
neurons_df.to_pickle("Metadata(auto).pkl")

neurons_df

,neuron_name,skeleton_id,nodes,connectors,tags
0,H-shaped _a1l,7766016,node_id parent_id creator_id ...,node_id connector_id type x...,"{'uncertain end': [17911173], 'not a branch': ..."
1,SOG IN left; PN into SOG very minor AL connect...,19431430,node_id parent_id creator_id ...,node_id connector_id type x...,"{'ends': [62933257, 14451713, 13948603, 629333..."
2,AN-R-Sens-B1-AVa-22,15564807,node_id parent_id creator_id ...,node_id connector_id type x ...,"{'mw axon split': [15767160], 'nerve ending': ..."
3,BAlp_ant ascending dendrite left,17383431,node_id parent_id creator_id ...,node_id connector_id type x...,"{'ends': [21655881, 63095422, 62977554, 618503..."
4,Drunken-4 a1l,7782409,node_id parent_id creator_id ...,node_id connector_id type x...,"{'ends': [12296747, 12297302, 12300044, 140977..."
...,...,...,...,...,...
5008,BAlp ant s 8 Right,17629170,node_id parent_id creator_id ...,"Empty DataFrame Columns: [node_id, connector_i...","{'soma': [23729370, 21817925], 'ends': [218180..."
5009,A02n_a1r Pseudolooper-4,2637812,node_id parent_id creator_id ...,node_id connector_id type x ...,"{'ends': [14648520, 14648527, 2828353, 3344731..."
5010,CP contra to SEZ left; MB1: incomplete neuron ...,3882998,node_id parent_id creator_id ...,node_id connector_id type x ...,"{'ends': [1745088, 1715337, 1715443, 1715596, ..."
5011,MN-L-Sens-B2-VM-08,15679484,node_id parent_id creator_id ...,node_id connector_id type x ...,"{'mw axon split': [16245025], 'Periphery': [16..."
